In [ ]:
%pip install -r requirements.txt
%config IPCompleter.greedy=True

In [ ]:
DEBUG = False
_doDebug = input('Debug mode [y/n]: ')
if _doDebug.lower() == 'y':
    DEBUG = True
elif _doDebug != 'n':
    print(f'response "{_doDebug}" not understood')

**1> collate workshop_id to mod_id(s) from source of truth collection**

In [ ]:
# Find and collate workshop_ids: mod_ids, store 
#data in an collection var
import requests
import re
from os import path

collection_id = '2903054110'
getCollections = 'https://api.steampowered.com/ISteamRemoteStorage/GetCollectionDetails/v1/?'
getFileDetails = 'https://api.steampowered.com/ISteamRemoteStorage/GetPublishedFileDetails/v1/?'

home_dir = path.expanduser('~')
print(f'user home dir:{home_dir}')

# collection contains the resulting list of tuples [(workshop_id, mod_id, mod_id, ...), ...] produced
# in this sell.
collection = list()

# raw_collection is a locally consumed var and contains the post-processed collection metadata response
raw_collection = list()
with open(path.join(home_dir, '.steam', 'token'), 'r') as f:
    auth_header = {'Authentication': f'Bearer {f.read().strip()}'}
    resp = requests.post(
        getCollections, 
        headers=auth_header,
        data={
            'publishedfileids[0]':f'{collection_id}',
            'collectioncount': '1',
        }
    )
    raw_collection = list(resp.json()['response']['collectiondetails'])


for child in raw_collection[0]['children']:
    child_id = child["publishedfileid"]
    resp = requests.post(
        getFileDetails,
        headers=auth_header,
        data={
            'itemcount': '1',
            'publishedfileids[0]': child_id,
        }
    )
    desc = resp.json()['response']['publishedfiledetails'][0]['description']
    match = re.findall(r'^Mod ID: .*', desc, flags=re.MULTILINE)
    if len(match) == 0:
        print(f'failed to find matching mod id string, mod file: {child_id}')
        break

    o = [child_id]
    for m in match:
        o.append(m)
    collection.append(tuple(o))
if len(collection) == 0:
    print('error')
    quit()
print(f'got upstream collection: {collection}')

**^^TODO create radial buttons per mod, per variant**

**2> Fetch the server config file**

In [ ]:
from os import path
from ftplib import FTP
from configparser import ConfigParser
import io
import re

nitrado_dir = path.join(home_dir, ".nitrado")
nitrado_ftp_url = 'usmi440.gamedata.io'
ftp_config = ConfigParser()
ftp_config.read(path.join(nitrado_dir, "ftp.ini"))

pz_cfg_src = '/zomboid/profile/Zomboid/Server/servertest.ini'

buf = io.BytesIO()
with FTP(nitrado_ftp_url) as ftp:
    print('logging into ftp server')
    ftp.login(
        user = ftp_config['DEFAULT']['username'], 
        passwd = ftp_config['DEFAULT']['password']
    )
    print('fetching servertest.ini')
    try:
        ftp.retrlines(f'LIST {pz_cfg_src}')
        ftp.retrbinary(f'RETR {pz_cfg_src}', buf.write)
    except Exception as e:
        print(e)
        quit()
    buf.seek(0)

**3> Merge the collection into the pz_config vars**

In [ ]:
import re

list_sep = ';'

# cur_[mods|wsi] contains the mods|WSIs currently enabled on the server
cur_mods = re.search(r'Mods=(.*)', buf.getvalue().decode()).group(0)
cur_mods = re.sub(r'Mods=(.*)', r'\1', cur_mods).split(list_sep)
cur_wsi = re.findall(r'WorkshopItems=([0-9;]*)', buf.getvalue().decode())[1].split(list_sep) # we only want the 2nd occurance of the string, first found is an example stub
print(f'Found current mods: {cur_mods}')
print(f'Found current WSIs: {cur_wsi}')

# Add new mods (i.e. mods not found in the config)
for c in collection:
    ws_item = c[0]
    mods = c[1:]
    print(f'processing: {ws_item}: {mods}')
    if ws_item not in cur_wsi:
        if ws_item.strip() == "":
            print('skipping null entry')
            continue

        print(f"{ws_item}({mods[0]}) not found, appending")
        cur_wsi.append(ws_item)
        cur_mods.append(mods[0])
    else:
        print(f'{ws_item} already added, skipping')
        continue
        
# Remove old mods (i.e. mods not found in the collection)
for i, wsi in enumerate(cur_wsi):
    # TODO this should not default to first mod variant.
    if wsi not in [o[0] for o in collection]:
        print(f'workshopitem {cur_wsi[i]}({cur_mods[i]}) not found in existing config, removing')
        del cur_wsi[i]
        del cur_mods[i]

print("Sync completed")
print(f'PROCESSED mods: {cur_mods}')
print(f'PROCESSED WSIs  : {cur_wsi}')

if len(cur_mods) is not len(cur_wsi):
    print(f'something went wrong, len(merge_mods)[{len(cur_mods)}] != len(merge_ws_items)[{len(cur_wsi)}]')
    quit(1)

**4> Write the pz config back to the ftp**

In [ ]:
import tempfile
from io import SEEK_SET
import os

list_sep=';'


mods_str = list_sep.join(cur_mods)
wsi_str = list_sep.join(cur_wsi)

print('updating in-memory config')
out_cfg = re.sub(fr'(\nMods=).*', repl=fr'\1{mods_str}', string=buf.getvalue().decode())
out_cfg = re.sub(fr'(\nWorkshopItems)=.*', repl=fr'\1={wsi_str}', string=out_cfg)

# print(f'out_cfg chunk: {out_cfg[12180:121100].encode()}')
if len(out_cfg.strip()) == 0:
    print('got an empty config string, bailing out')
    quit(1)

with FTP(nitrado_ftp_url) as ftp:
    print('writing out config')
    ftp.login(
        user = ftp_config['DEFAULT']['username'],
        passwd = ftp_config['DEFAULT']['password']
    )
    out_buf = io.BytesIO()
    out_buf.write(out_cfg.encode(encoding='utf-8'))
    out_buf.seek(0, SEEK_SET)
    if DEBUG:
        # print(f'out_buf:\n{out_buf.getvalue()}')
        with open('servertest.ini', 'w', encoding='utf-8') as f:
            f.write(out_buf.getvalue().decode())
        print('DEBUG was enabled, ending processing here')
    elif len(out_buf.getvalue().strip()) == 0:
        print('generated busted file, quitting without writing')
    else:
        print(f'Mods={mods_str}')
        print(f'WorkshopItems={wsi_str}')
        def encodeLine(l:str) -> bytes:
            return l.encode(encoding='utf-8')
        print('transmitting to ftp remote')
        ftp.storlines(cmd=f'STOR {pz_cfg_src}', fp=out_buf)